***
<font size="6"><center><b> H-CapsNet: A Capsule Network for Hierarchical Image Classification </b></center></font>
***

# Model Description

**H-CapsNet model:**
- Use Capsule network for hierarchcial classification
- This model contains a deducated feature extraction layer and Capsule netowrk per hierarchy
- For training use MixupData data augmentation technique
- Model Uses Dynamic LossWeight Distribution system
- This model is designed and evaluted using **TensorFlow 2.8.0**

# Import necessary Files and Libraries

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import regularizers, optimizers
from tensorflow.keras import backend as K
# Supporting Libraries:
    #Mathplot lib for ploting graphs
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
    # numpy and pandas
import numpy as np
import pandas as pd
    #system
import os
import sys
import csv
    #import other libraries
import math
import random
from datetime import datetime
    # ML model, Dataset and evalution metrics
from src import datasets
from src import MLmodel
from src import metrics
from src import MixUp
from src import sysenv
    # For developind (reloades any python scripts)
import importlib

%load_ext autoreload
%autoreload 2

# System information & GPU growth

In [ ]:
systeminfo = sysenv.systeminfo()
print(systeminfo)

In [ ]:
## For Using Multiple GPUs
gpus = "0" ## Selecting Available gpus
gpugrowth = sysenv.gpugrowth(gpus = gpus) ## Limiting GPUS from OS environment
gpugrowth.memory_growth() #GPU memory growth

# Training parameters

## Parameters

In [ ]:
train_params = {"n_epochs" : 100,
                "batch_size": 32,
                "lr": 0.001, # Initial learning rate
                "lr_decay": 0.95, # Learning rate decay
                "decay_exe": 9, #learning rate decay execution epoch after
               }
model_params = {"P_Cap_Dim" : 8, # Primary Capsule Dimentions
                "S_Cap_Dim" : 16, # Secondary Capsule Dimention
                "Reconstruction_LW" : 0.0005, # Decoder loss weight
                "class_loss" : MLmodel.MarginLoss(), ## Class prediction loss
                "reconstruction_loss" : 'mse'
               }

## Learning rate decay

In [ ]:
def scheduler(epoch):
    learning_rate_init = train_params["lr"]
    
    if epoch > train_params["decay_exe"]:
        learning_rate_init = train_params["lr"] * (train_params["lr_decay"] ** (epoch-9))
        
    return learning_rate_init

# MNIST dataset

## Import MNIST dataset

In [ ]:
## Import dataset
dataset = datasets.MNIST()
# dataset = datasets.MNIST(version = 'reduce')
print(dataset.keys())
dataset['tree'].show()


input_shape = dataset['x_train'].shape[1:]
print('INPUT SHAPE:',input_shape,'\n')

print("TRAIN: \r\n")
print(dataset['x_train'].shape)
print(dataset['y_train_fine'].shape)
print(dataset['y_train_coarse'].shape)

print("\nTEST: \r\n")
print(dataset['x_test'].shape)
print(dataset['y_test_fine'].shape)
print(dataset['y_test_coarse'].shape)

fine_class = len(np.unique(np.argmax(dataset['y_test_fine'], axis=1)))
coarse_class = len(np.unique(np.argmax(dataset['y_test_coarse'], axis=1)))
print('\nNumber of Classes in Label Tree:',
      '\nCoarse Level = ',coarse_class,
      '\nFine Level = ',fine_class)

# Plot Random samples 
datasets.plot_sample_image(dataset['x_train'],
                           {'coarse':dataset['y_train_coarse'],
                            'fine':dataset['y_train_fine']})

## Create ML Model

### Model parameters

In [ ]:
initial_lw = MLmodel.initial_lw({"coarse": coarse_class, "fine": fine_class})

lossweight = {'coarse_lw' : K.variable(value = initial_lw['coarse'], dtype="float32", name="coarse_lw"),
             'fine_lw' : K.variable(value = initial_lw['fine'], dtype="float32", name="fine_lw"),
              'decoder_lw' : model_params['Reconstruction_LW']
             }

def get_compiled_model():
    model = MLmodel.HCapsNet_2_Level_MNIST(input_shape,
                                   coarse_class, fine_class)
    model.compile(optimizer='adam',
                  loss=[model_params["class_loss"],
                        model_params["class_loss"],
                        model_params["reconstruction_loss"]],
                  loss_weights=[lossweight['coarse_lw'],
                                lossweight['fine_lw'],
                                lossweight['decoder_lw']],
                  metrics={'Fine_prediction_output_layer': 'accuracy',
                           'Coarse_prediction_output_layer': 'accuracy'}
                 )
    return model

In [ ]:
tf.keras.backend.clear_session() ## clear session

# model = get_compiled_model()

#Multiple GPU ------ Windows
strategy = tf.distribute.MirroredStrategy(cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())

print("Number of devices: {}".format(strategy.num_replicas_in_sync))

# Open a strategy scope.
with strategy.scope():
    # Everything that creates variables should be under the strategy scope.
    # In general this is only model construction & `compile()`.
    model = get_compiled_model()

### Log directory
directory = sysenv.log_dir(dataset["name"]+'/'+model.name)

model.summary()
keras.utils.plot_model(model, to_file = directory+"/H-CapsNet.png", show_shapes=True)

### Callbacks

In [ ]:
tb = keras.callbacks.TensorBoard(directory+'./tb_logs')
log = keras.callbacks.CSVLogger(directory+'/log.csv', append=True)

checkpoint = keras.callbacks.ModelCheckpoint(directory+'/epoch_best.h5',
                                             monitor='val_Fine_prediction_output_layer_accuracy',
                                             save_best_only=True, save_weights_only=True, verbose=1
                                            )

change_lw = MLmodel.LossWeightsModifier(lossweight,
                                        initial_lw,
                                        directory = directory)

lr_decay = keras.callbacks.LearningRateScheduler(scheduler)

## Training Model

In [ ]:
datagen = ImageDataGenerator(width_shift_range=0.1,
                            height_shift_range=0.1)

training_generator = MixUp.MixupGenerator_2level(dataset['x_train'],
                                                 dataset['y_train_coarse'], dataset['y_train_fine'],
                                                 batch_size=train_params["batch_size"], alpha=0.02, 
                                                 datagen=datagen
                                                )()

In [ ]:
history = model.fit_generator(generator=training_generator,
                                  steps_per_epoch = int(dataset['x_train'].shape[0] / train_params["batch_size"]),
                                  epochs = train_params["n_epochs"],
                                  validation_data = ([dataset['x_test'], dataset['y_test_coarse'], dataset['y_test_fine']],
                                                     [dataset['y_test_coarse'], dataset['y_test_fine'], dataset['x_test']]),
                                  callbacks = [tb, log, checkpoint, lr_decay, change_lw],
                                  verbose=1)
model_save_dir = str(directory+'/trained_model.h5')
try:
    model.save_weights(model_save_dir)
    print('Trained model saved to = ', model_save_dir)
except:
    print('Model Wight is not saved')

## Model Analysis

In [ ]:
model_analysis = MLmodel.model_analysis(model, dataset)
results = model_analysis.evaluate()
predictions = model_analysis.prediction()

true_label = [dataset['y_test_coarse'],dataset['y_test_fine']]
pred_label = [predictions[0],predictions[1]]
metrics.lvl_wise_metric(true_label,pred_label)

h_measurements,consistency,exact_match = metrics.hmeasurements(true_label,
                                       pred_label,
                                       dataset['tree'])
print('\nHierarchical Precision =',h_measurements[0],
      '\nHierarchical Recall =', h_measurements[1],
      '\nHierarchical F1-Score =',h_measurements[2],
      '\n Consistency = ', consistency,
      '\n Exact Match = ', exact_match,
     )

# EMNIST Dataset

## Import EMNIST dataset

In [ ]:
## Import dataset
dataset = datasets.E_MNIST()
# dataset = datasets.E_MNIST(version='reduce')
print(dataset.keys())
dataset['tree'].show()

input_shape = dataset['x_train'].shape[1:]
print('INPUT SHAPE:',input_shape,'\n')

print("TRAIN: \r\n")
print(dataset['x_train'].shape)
print(dataset['y_train_fine'].shape)
print(dataset['y_train_coarse'].shape)

print("\nTEST: \r\n")
print(dataset['x_test'].shape)
print(dataset['y_test_fine'].shape)
print(dataset['y_train_coarse'].shape)

fine_class = len(np.unique(np.argmax(dataset['y_train_fine'], axis=1)))
coarse_class = len(np.unique(np.argmax(dataset['y_train_coarse'], axis=1)))
print('\nNumber of Classes in Label Tree:',
      '\nCoarse Level = ',coarse_class,
      '\nFine Level = ',fine_class)
    # Plot Random samples 
datasets.plot_sample_image(dataset['x_train'],
                           {'coarse':dataset['y_train_coarse'],
                            'fine':dataset['y_train_fine']})

## Create ML Model

### Model parameters

In [ ]:
initial_lw = MLmodel.initial_lw({"coarse": coarse_class, "fine": fine_class})

lossweight = {'coarse_lw' : K.variable(value = initial_lw['coarse'], dtype="float32", name="coarse_lw"),
             'fine_lw' : K.variable(value = initial_lw['fine'], dtype="float32", name="fine_lw"),
              'decoder_lw' : model_params['Reconstruction_LW']
             }

def get_compiled_model():
    model = MLmodel.HCapsNet_2_Level_EMNIST(input_shape,
                                   coarse_class, fine_class)
    model.compile(optimizer='adam',
                  loss=[model_params["class_loss"],
                        model_params["class_loss"],
                        model_params["reconstruction_loss"]],
                  loss_weights=[lossweight['coarse_lw'],
                                lossweight['fine_lw'],
                                lossweight['decoder_lw']],
                  metrics={'Fine_prediction_output_layer': 'accuracy',
                           'Coarse_prediction_output_layer': 'accuracy'}
                 )
    return model

In [ ]:
tf.keras.backend.clear_session() ## clear session

# model = get_compiled_model()

#Multiple GPU ------ Windows
strategy = tf.distribute.MirroredStrategy(cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())

print("Number of devices: {}".format(strategy.num_replicas_in_sync))

# Open a strategy scope.
with strategy.scope():
    # Everything that creates variables should be under the strategy scope.
    # In general this is only model construction & `compile()`.
    model = get_compiled_model()

### Log directory
directory = sysenv.log_dir(dataset["name"]+'/'+model.name)

model.summary()
keras.utils.plot_model(model, to_file = directory+"/H-CapsNet.png", show_shapes=True)

### Callbacks

In [ ]:
tb = keras.callbacks.TensorBoard(directory+'./tb_logs')
log = keras.callbacks.CSVLogger(directory+'/log.csv', append=True)

checkpoint = keras.callbacks.ModelCheckpoint(directory+'/epoch_best.h5',
                                             monitor='val_Fine_prediction_output_layer_accuracy',
                                             save_best_only=True, save_weights_only=True, verbose=1
                                            )

change_lw = MLmodel.LossWeightsModifier(lossweight,
                                        initial_lw,
                                        directory = directory)
lr_decay = keras.callbacks.LearningRateScheduler(scheduler)

## Training Model

In [ ]:
datagen = ImageDataGenerator(width_shift_range=0.1,
                            height_shift_range=0.1)

training_generator = MixUp.MixupGenerator_2level(dataset['x_train'],
                                                 dataset['y_train_coarse'], dataset['y_train_fine'],
                                                 batch_size=train_params["batch_size"], alpha=0.02, 
                                                 datagen=datagen
                                                )()

In [ ]:
history = model.fit_generator(generator=training_generator,
                                  steps_per_epoch = int(dataset['x_train'].shape[0] / train_params["batch_size"]),
                                  epochs = train_params["n_epochs"],
                                  validation_data = ([dataset['x_test'], dataset['y_test_coarse'], dataset['y_test_fine']],
                                                     [dataset['y_test_coarse'], dataset['y_test_fine'], dataset['x_test']]),
                                  callbacks = [tb, log, checkpoint, lr_decay, change_lw],
                                  verbose=1)
model_save_dir = str(directory+'/trained_model.h5')
try:
    model.save_weights(model_save_dir)
    print('Trained model saved to = ', model_save_dir)
except:
    print('Model Wight is not saved')

## Model Analysis

In [ ]:
model_analysis = MLmodel.model_analysis(model, dataset)
results = model_analysis.evaluate()
predictions = model_analysis.prediction()

true_label = [dataset['y_test_coarse'],dataset['y_test_fine']]
pred_label = [predictions[0],predictions[1]]
metrics.lvl_wise_metric(true_label,pred_label)

h_measurements,consistency,exact_match = metrics.hmeasurements(true_label,
                                       pred_label,
                                       dataset['tree'])
print('\nHierarchical Precision =',h_measurements[0],
      '\nHierarchical Recall =', h_measurements[1],
      '\nHierarchical F1-Score =',h_measurements[2],
      '\n Consistency = ', consistency,
      '\n Exact Match = ', exact_match,
     )

# Fashion-MNIST dataset

## Import Fashion-MNIST dataset

In [ ]:
## Import dataset
dataset = datasets.F_MNIST()
# dataset = datasets.F_MNIST('reduce')
print(dataset.keys())
dataset['tree'].show()

input_shape = dataset['x_train'].shape[1:]
print('INPUT SHAPE:',input_shape,'\n')

print("TRAIN: \r\n")
print(dataset['x_train'].shape)
print(dataset['y_train_fine'].shape)
print(dataset['y_train_medium'].shape)
print(dataset['y_train_coarse'].shape)

print("\nTEST: \r\n")
print(dataset['x_test'].shape)
print(dataset['y_test_fine'].shape)
print(dataset['y_test_medium'].shape)
print(dataset['y_test_coarse'].shape)

fine_class = len(np.unique(np.argmax(dataset['y_train_fine'], axis=1)))
medium_class = len(np.unique(np.argmax(dataset['y_train_medium'], axis=1)))
coarse_class = len(np.unique(np.argmax(dataset['y_train_coarse'], axis=1)))
print('\nNumber of Classes in Label Tree:',
      '\nCoarse Level = ',coarse_class,
      '\nMedium Level = ',medium_class,
      '\nFine Level = ',fine_class)

datasets.plot_sample_image(dataset['x_train'],
                           {'coarse':dataset['y_train_coarse'],
                            'medium':dataset['y_train_medium'],
                            'fine':dataset['y_train_fine']})

### Model parameters

In [ ]:
initial_lw = MLmodel.initial_lw({"coarse": coarse_class,
                                 "medium": medium_class,
                                 "fine": fine_class})

lossweight = {'coarse_lw' : K.variable(value = initial_lw['coarse'], dtype="float32", name="coarse_lw"),
             'medium_lw' : K.variable(value = initial_lw['medium'], dtype="float32", name="medium_lw"),
             'fine_lw' : K.variable(value = initial_lw['fine'], dtype="float32", name="fine_lw"),
              'decoder_lw' : model_params['Reconstruction_LW']
             }

def get_compiled_model():
    model = MLmodel.HCapsNet_3_Level_FMNIST(input_shape,
                                     coarse_class, medium_class, fine_class)
    model.compile(optimizer='adam',
                  loss=[model_params["class_loss"],
                        model_params["class_loss"],
                        model_params["class_loss"],
                        model_params["reconstruction_loss"]],
                  loss_weights=[lossweight['coarse_lw'],
                                lossweight['medium_lw'],
                                lossweight['fine_lw'],
                                lossweight['decoder_lw']],
                  metrics={'Fine_prediction_output_layer': 'accuracy',
                           'Medium_prediction_output_layer': 'accuracy',
                           'Coarse_prediction_output_layer': 'accuracy'}
                 )
    return model

In [ ]:
tf.keras.backend.clear_session() ## clear session

# model = get_compiled_model()

#Multiple GPU ------ Windows
strategy = tf.distribute.MirroredStrategy(cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())

print("Number of devices: {}".format(strategy.num_replicas_in_sync))

# Open a strategy scope.
with strategy.scope():
    # Everything that creates variables should be under the strategy scope.
    # In general this is only model construction & `compile()`.
    model = get_compiled_model()

### Log directory
directory = sysenv.log_dir(dataset["name"]+'/'+model.name)

model.summary()
keras.utils.plot_model(model, to_file = directory+"/H-CapsNet.png", show_shapes=True)

### Callbacks

In [ ]:
tb = keras.callbacks.TensorBoard(directory+'./tb_logs')
log = keras.callbacks.CSVLogger(directory+'/log.csv', append=True)

checkpoint = keras.callbacks.ModelCheckpoint(directory+'/epoch_best.h5',
                                             monitor='val_Fine_prediction_output_layer_accuracy',
                                             save_best_only=True, save_weights_only=True, verbose=1
                                            )

change_lw = MLmodel.LossWeightsModifier(lossweight,
                                        initial_lw,
                                        directory = directory)
lr_decay = keras.callbacks.LearningRateScheduler(scheduler)

## Training Model

In [ ]:
datagen = ImageDataGenerator(width_shift_range=0.1,
                            height_shift_range=0.1)

training_generator = MixUp.MixupGenerator_3level(dataset['x_train'],
                                                 dataset['y_train_coarse'], 
                                                 dataset['y_train_medium'],
                                                 dataset['y_train_fine'],
                                                 batch_size=train_params["batch_size"],
                                                 alpha=0.2, 
                                                 datagen=datagen
                                                )()

In [ ]:
history = model.fit_generator(generator=training_generator,
                              steps_per_epoch = int(dataset['x_train'].shape[0] / train_params["batch_size"]),
                              epochs = train_params["n_epochs"],
                              validation_data = ([dataset['x_test'],
                                                  dataset['y_test_coarse'],
                                                  dataset['y_test_medium'],
                                                  dataset['y_test_fine']],
                                                 [dataset['y_test_coarse'],
                                                  dataset['y_test_medium'],
                                                  dataset['y_test_fine'],
                                                  dataset['x_test']]),
                                  callbacks = [tb,
                                               log,
                                               checkpoint,
                                               lr_decay,
                                               change_lw],
                                  verbose=1)
model_save_dir = str(directory+'/trained_model.h5')
try:
    model.save_weights(model_save_dir)
    print('Trained model saved to = ', model_save_dir)
except:
    print('Model Wight is not saved')

## Model Analysis

In [ ]:
model_analysis = MLmodel.model_analysis(model, dataset)

results = model_analysis.evaluate()

predictions = model_analysis.prediction()

true_label = [dataset['y_test_coarse'],dataset['y_test_medium'],dataset['y_test_fine']]
pred_label = [predictions[0],predictions[1],predictions[2]]
metrics.lvl_wise_metric(true_label,pred_label)

h_measurements,consistency,exact_match = metrics.hmeasurements(true_label,
                                       pred_label,
                                       dataset['tree'])
print('\nHierarchical Precision =',h_measurements[0],
      '\nHierarchical Recall =', h_measurements[1],
      '\nHierarchical F1-Score =',h_measurements[2],
      '\n Consistency = ', consistency,
      '\n Exact Match = ', exact_match,
     )

# CIFAR-10 Dataset

## Import CIFAR-10 dataset

In [ ]:
## Import dataset
dataset = datasets.CIFAR10()
# dataset = datasets.CIFAR10('reduce')
print(dataset.keys())
dataset['tree'].show()

input_shape = dataset['x_train'].shape[1:]
print('INPUT SHAPE:',input_shape,'\n')

print("TRAIN: \r\n")
print(dataset['x_train'].shape)
print(dataset['y_train_fine'].shape)
print(dataset['y_train_medium'].shape)
print(dataset['y_train_coarse'].shape)

print("\nTEST: \r\n")
print(dataset['x_test'].shape)
print(dataset['y_test_fine'].shape)
print(dataset['y_test_medium'].shape)
print(dataset['y_test_coarse'].shape)

fine_class = len(np.unique(np.argmax(dataset['y_train_fine'], axis=1)))
medium_class = len(np.unique(np.argmax(dataset['y_train_medium'], axis=1)))
coarse_class = len(np.unique(np.argmax(dataset['y_train_coarse'], axis=1)))
print('\nNumber of Classes in Label Tree:',
      '\nCoarse Level = ',coarse_class,
      '\nMedium Level = ',medium_class,
      '\nFine Level = ',fine_class)
datasets.plot_sample_image(dataset['x_train'],
                           {'coarse':dataset['y_train_coarse'],
                            'medium':dataset['y_train_medium'],
                            'fine':dataset['y_train_fine']})

### Model parameters

In [ ]:
initial_lw = MLmodel.initial_lw({"coarse": coarse_class,
                                 "medium": medium_class,
                                 "fine": fine_class})

lossweight = {'coarse_lw' : K.variable(value = initial_lw['coarse'], dtype="float32", name="coarse_lw"),
             'medium_lw' : K.variable(value = initial_lw['medium'], dtype="float32", name="medium_lw"),
             'fine_lw' : K.variable(value = initial_lw['fine'], dtype="float32", name="fine_lw"),
              'decoder_lw' : model_params['Reconstruction_LW']
             }

def get_compiled_model():
    model = MLmodel.HCapsNet_3_Level(input_shape,
                                     coarse_class, medium_class, fine_class)
    model.compile(optimizer='adam',
                  loss=[model_params["class_loss"],
                        model_params["class_loss"],
                        model_params["class_loss"],
                        model_params["reconstruction_loss"]],
                  loss_weights=[lossweight['coarse_lw'],
                                lossweight['medium_lw'],
                                lossweight['fine_lw'],
                                lossweight['decoder_lw']],
                  metrics={'Fine_prediction_output_layer': 'accuracy',
                           'Medium_prediction_output_layer': 'accuracy',
                           'Coarse_prediction_output_layer': 'accuracy'}
                 )
    return model

In [ ]:
tf.keras.backend.clear_session() ## clear session

# model = get_compiled_model()

#Multiple GPU ------ Windows
strategy = tf.distribute.MirroredStrategy(cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())

print("Number of devices: {}".format(strategy.num_replicas_in_sync))

# Open a strategy scope.
with strategy.scope():
    # Everything that creates variables should be under the strategy scope.
    # In general this is only model construction & `compile()`.
    model = get_compiled_model()

### Log directory
directory = sysenv.log_dir(dataset["name"]+'/'+model.name)

model.summary()
keras.utils.plot_model(model, to_file = directory+"/H-CapsNet.png", show_shapes=True)

### Callbacks

In [ ]:
tb = keras.callbacks.TensorBoard(directory+'./tb_logs')
log = keras.callbacks.CSVLogger(directory+'/log.csv', append=True)

checkpoint = keras.callbacks.ModelCheckpoint(directory+'/epoch_best.h5',
                                             monitor='val_Fine_prediction_output_layer_accuracy',
                                             save_best_only=True, save_weights_only=True, verbose=1
                                            )

change_lw = MLmodel.LossWeightsModifier(lossweight,
                                        initial_lw,
                                        directory = directory)

lr_decay = keras.callbacks.LearningRateScheduler(scheduler)

## Training Model

In [ ]:
datagen = ImageDataGenerator(width_shift_range=0.1,
                            height_shift_range=0.1)

training_generator = MixUp.MixupGenerator_3level(dataset['x_train'],
                                                 dataset['y_train_coarse'], 
                                                 dataset['y_train_medium'],
                                                 dataset['y_train_fine'],
                                                 batch_size=train_params["batch_size"],
                                                 alpha=0.2, 
                                                 datagen=datagen
                                                )()

In [ ]:
history = model.fit_generator(generator=training_generator,
                              steps_per_epoch = int(dataset['x_train'].shape[0] / train_params["batch_size"]),
                              epochs = train_params["n_epochs"],
                              validation_data = ([dataset['x_test'],
                                                  dataset['y_test_coarse'],
                                                  dataset['y_test_medium'],
                                                  dataset['y_test_fine']],
                                                 [dataset['y_test_coarse'],
                                                  dataset['y_test_medium'],
                                                  dataset['y_test_fine'],
                                                  dataset['x_test']]),
                                  callbacks = [tb,
                                               log,
                                               checkpoint,
                                               lr_decay,
                                               change_lw],
                                  verbose=1)
model_save_dir = str(directory+'/trained_model.h5')
try:
    model.save_weights(model_save_dir)
    print('Trained model saved to = ', model_save_dir)
except:
    print('Model Wight is not saved')

## Model Analysis

In [ ]:
model_analysis = MLmodel.model_analysis(model, dataset)

results = model_analysis.evaluate()

predictions = model_analysis.prediction()

true_label = [dataset['y_test_coarse'],dataset['y_test_medium'],dataset['y_test_fine']]
pred_label = [predictions[0],predictions[1],predictions[2]]
metrics.lvl_wise_metric(true_label,pred_label)

h_measurements,consistency,exact_match = metrics.hmeasurements(true_label,
                                       pred_label,
                                       dataset['tree'])
print('\nHierarchical Precision =',h_measurements[0],
      '\nHierarchical Recall =', h_measurements[1],
      '\nHierarchical F1-Score =',h_measurements[2],
      '\n Consistency = ', consistency,
      '\n Exact Match = ', exact_match,
     )

# CIFAR-100 Dataset

## Import CIFAR-100 dataset

In [ ]:
## Import dataset
dataset = datasets.CIFAR100()
# dataset = datasets.CIFAR100('reduce')
print(dataset.keys())
dataset['tree'].show()

input_shape = dataset['x_train'].shape[1:]
print('INPUT SHAPE:',input_shape,'\n')

print("TRAIN: \r\n")
print(dataset['x_train'].shape)
print(dataset['y_train_fine'].shape)
print(dataset['y_train_medium'].shape)
print(dataset['y_train_coarse'].shape)

print("\nTEST: \r\n")
print(dataset['x_test'].shape)
print(dataset['y_test_fine'].shape)
print(dataset['y_test_medium'].shape)
print(dataset['y_test_coarse'].shape)

fine_class = len(np.unique(np.argmax(dataset['y_train_fine'], axis=1)))
medium_class = len(np.unique(np.argmax(dataset['y_train_medium'], axis=1)))
coarse_class = len(np.unique(np.argmax(dataset['y_train_coarse'], axis=1)))
print('\nNumber of Classes in Label Tree:',
      '\nCoarse Level = ',coarse_class,
      '\nMedium Level = ',medium_class,
      '\nFine Level = ',fine_class)

datasets.plot_sample_image(dataset['x_train'],
                           {'coarse':dataset['y_train_coarse'],
                            'medium':dataset['y_train_medium'],
                            'fine':dataset['y_train_fine']})

### Model parameters

In [ ]:
initial_lw = MLmodel.initial_lw({"coarse": coarse_class,
                                 "medium": medium_class,
                                 "fine": fine_class})

lossweight = {'coarse_lw' : K.variable(value = initial_lw['coarse'], dtype="float32", name="coarse_lw"),
             'medium_lw' : K.variable(value = initial_lw['medium'], dtype="float32", name="medium_lw"),
             'fine_lw' : K.variable(value = initial_lw['fine'], dtype="float32", name="fine_lw"),
              'decoder_lw' : model_params['Reconstruction_LW']
             }

def get_compiled_model():
    model = MLmodel.HCapsNet_3_Level(input_shape,
                                     coarse_class, medium_class, fine_class)
    model.compile(optimizer='adam',
                  loss=[model_params["class_loss"],
                        model_params["class_loss"],
                        model_params["class_loss"],
                        model_params["reconstruction_loss"]],
                  loss_weights=[lossweight['coarse_lw'],
                                lossweight['medium_lw'],
                                lossweight['fine_lw'],
                                lossweight['decoder_lw']],
                  metrics={'Fine_prediction_output_layer': 'accuracy',
                           'Medium_prediction_output_layer': 'accuracy',
                           'Coarse_prediction_output_layer': 'accuracy'}
                 )
    return model

In [ ]:
tf.keras.backend.clear_session() ## clear session

# model = get_compiled_model()

#Multiple GPU ------ Windows
strategy = tf.distribute.MirroredStrategy(cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())

print("Number of devices: {}".format(strategy.num_replicas_in_sync))

# Open a strategy scope.
with strategy.scope():
    # Everything that creates variables should be under the strategy scope.
    # In general this is only model construction & `compile()`.
    model = get_compiled_model()

### Log directory
directory = sysenv.log_dir(dataset["name"]+'/'+model.name)

model.summary()
keras.utils.plot_model(model, to_file = directory+"/H-CapsNet.png", show_shapes=True)

### Callbacks

In [ ]:
tb = keras.callbacks.TensorBoard(directory+'./tb_logs')

log = keras.callbacks.CSVLogger(directory+'/log.csv', append=True)

checkpoint = keras.callbacks.ModelCheckpoint(directory+'/epoch_best.h5',
                                             monitor='val_Fine_prediction_output_layer_accuracy',
                                             save_best_only=True, save_weights_only=True, verbose=1
                                            )

change_lw = MLmodel.LossWeightsModifier(lossweight,
                                        initial_lw,
                                        directory = directory)

lr_decay = keras.callbacks.LearningRateScheduler(scheduler)

## Training Model

In [ ]:
datagen = ImageDataGenerator(width_shift_range=0.1,
                            height_shift_range=0.1)

training_generator = MixUp.MixupGenerator_3level(dataset['x_train'],
                                                 dataset['y_train_coarse'], 
                                                 dataset['y_train_medium'],
                                                 dataset['y_train_fine'],
                                                 batch_size=train_params["batch_size"],
                                                 alpha=0.2, 
                                                 datagen=datagen
                                                )()

In [ ]:
history = model.fit_generator(generator=training_generator,
                              steps_per_epoch = int(dataset['x_train'].shape[0] / train_params["batch_size"]),
                              epochs = train_params["n_epochs"],
                              validation_data = ([dataset['x_test'],
                                                  dataset['y_test_coarse'],
                                                  dataset['y_test_medium'],
                                                  dataset['y_test_fine']],
                                                 [dataset['y_test_coarse'],
                                                  dataset['y_test_medium'],
                                                  dataset['y_test_fine'],
                                                  dataset['x_test']]),
                                  callbacks = [tb,
                                               log,
                                               checkpoint,
                                               lr_decay,
                                               change_lw],
                                  verbose=1)
model_save_dir = str(directory+'/trained_model.h5')
try:
    model.save_weights(model_save_dir)
    print('Trained model saved to = ', model_save_dir)
except:
    print('Model Wight is not saved')

## Model Analysis

In [ ]:
model_analysis = MLmodel.model_analysis(model, dataset)

results = model_analysis.evaluate()

predictions = model_analysis.prediction()

true_label = [dataset['y_test_coarse'],dataset['y_test_medium'],dataset['y_test_fine']]
pred_label = [predictions[0],predictions[1],predictions[2]]
metrics.lvl_wise_metric(true_label,pred_label)

h_measurements,consistency,exact_match = metrics.hmeasurements(true_label,
                                       pred_label,
                                       dataset['tree'])
print('\nHierarchical Precision =',h_measurements[0],
      '\nHierarchical Recall =', h_measurements[1],
      '\nHierarchical F1-Score =',h_measurements[2],
      '\n Consistency = ', consistency,
      '\n Exact Match = ', exact_match,
     )

# Marine-tree Dataset

## Import Dataset

In [ ]:
dataset = datasets.get_Marine_dataset(output_level='level_depth_3',
                                      dataset_path ='D:\Datasets\Marine_tree',
                                      image_size=(64,64),
                                      batch_size=train_params['batch_size'],
                                      subtype='Combined',
                                      data_normalizing ='normalize',
                                      class_encoding = 'One_Hot_Encoder',
                                      data_augmantation = 'mixup'
                                     )

In [ ]:
for x,y in dataset.train_dataset.take(1):
    for i in range(len(x)):
        print('Example = ', i)
        plt.imshow(x[i])
        plt.show()
        print('Coarse =', {k:v for k,v in enumerate(y[0][i].numpy()) if v != 0}) # coarse lables
        print('Medium =', {k:v for k,v in enumerate(y[1][i].numpy()) if v != 0}) # medium lables
        print('Fine   =', {k:v for k,v in enumerate(y[2][i].numpy()) if v != 0}) # fine lables

## Model parameters

In [ ]:
coarse_class, medium_class, fine_class = dataset.num_classes

initial_lw = MLmodel.initial_lw({"coarse": coarse_class,
                                 "medium": medium_class,
                                 "fine": fine_class})

lossweight = {'coarse_lw' : K.variable(value = initial_lw['coarse'], dtype="float32", name="coarse_lw"),
             'medium_lw' : K.variable(value = initial_lw['medium'], dtype="float32", name="medium_lw"),
             'fine_lw' : K.variable(value = initial_lw['fine'], dtype="float32", name="fine_lw"),
              'decoder_lw' : model_params['Reconstruction_LW']
             }

## Get Model

In [ ]:
def get_compiled_model():
    model = MLmodel.HCapsNet_3_Level(input_shape = dataset.image_size,
                                        no_coarse_class  = coarse_class,
                                        no_medium_class  = medium_class,
                                        no_fine_class    = fine_class
                                       )
    model.compile(optimizer='adam',
                  loss=[model_params["class_loss"],
                        model_params["class_loss"],
                        model_params["class_loss"],
                        model_params["reconstruction_loss"]],
                  loss_weights=[lossweight['coarse_lw'],
                                lossweight['medium_lw'],
                                lossweight['fine_lw'],
                                lossweight['decoder_lw']],
                  metrics={'Fine_prediction_output_layer': 'accuracy',
                           'Medium_prediction_output_layer': 'accuracy',
                           'Coarse_prediction_output_layer': 'accuracy'}
                 )
    return model

In [ ]:
tf.keras.backend.clear_session() ## clear session

In [ ]:
model = get_compiled_model()

## Log Directory

In [ ]:
### Log directory
directory = sysenv.log_dir(dataset.name+'/'+model.name)

## Model Summary

In [ ]:
model.summary()
keras.utils.plot_model(model, to_file = directory+"/H-CapsNet_Marine_tree.png", show_shapes=True)

## Callbacks

In [ ]:
tb = keras.callbacks.TensorBoard(directory+'/tb_logs')
log = keras.callbacks.CSVLogger(directory+'/log.csv', append=True)

checkpoint = keras.callbacks.ModelCheckpoint(directory+'/epoch_best.h5',
                                             monitor='val_Fine_prediction_output_layer_accuracy',
                                             save_best_only=True, save_weights_only=True, verbose=1
                                            )

change_lw = MLmodel.LossWeightsModifier(lossweight,
                                        initial_lw,
                                        directory = directory)

lr_decay = keras.callbacks.LearningRateScheduler(scheduler)

## Dataset Training Pipeline

In [ ]:
def pipeline_multi_input_output(image, label):
    label_0 = label[0]
    label_1 = label[1]
    label_2 = label[2]
    return (image, label_0, label_1, label_2), (label_0, label_1, label_2, image)

### matchin X,Y with model input
training_dataset_match = dataset.train_dataset.map(pipeline_multi_input_output) ## Mixup dataset
val_dataset_match = dataset.val_dataset.map(pipeline_multi_input_output) ## Val Dataset
test_dataset_match = dataset.test_dataset.map(pipeline_multi_input_output) ## test Dataset

## Training Model

In [ ]:
history = model.fit(training_dataset_match,
                    batch_size = train_params["batch_size"],
                    epochs = train_params["n_epochs"],
                    validation_data = val_dataset_match,
                    callbacks = [tb,
                                 log,
                                 checkpoint,
                                 lr_decay,
                                 change_lw],
                    verbose=1)
model_save_dir = str(directory+'/trained_model.h5')
try:
    model.save_weights(model_save_dir)
    print('Trained model saved to = ', model_save_dir)
except:
    print('Model Wight is not saved')

In [ ]:
try:
    history_dict = history.history
    plotter = tfdocs.plots.HistoryPlotter()
    plotter.plot({"Coarse": history}, metric = "Coarse_prediction_output_layer_accuracy")
    plotter.plot({"Medium": history}, metric = "Medium_prediction_output_layer_accuracy")
    plotter.plot({"Fine": history}, metric = "Fine_prediction_output_layer_accuracy")
    plt.title("Model Accuracy")
    plt.ylim([0,1])
except:
    print('Not available')

## Model Analysis

In [ ]:
results = model.evaluate(test_dataset_match)
for n in range(len(results)):
    print(str(n+1)+'.',model.metrics_names[n], '==>', results[n])

In [ ]:
def predict_from_pipeline(model, dataset):
    y_pred_c = []
    y_pred_m = []
    y_pred_f = []
    
    y_true_c = []
    y_true_m = []
    y_true_f = []
    for x, y in dataset:
        batch_pred = model.predict(x)
        
        y_true_c.extend(y[0].numpy().tolist())
        y_true_m.extend(y[1].numpy().tolist())
        y_true_f.extend(y[2].numpy().tolist())
        
        y_pred_c.extend(batch_pred[0].tolist())
        y_pred_m.extend(batch_pred[1].tolist())
        y_pred_f.extend(batch_pred[2].tolist())
        
    return np.array(y_true_c), np.array(y_true_m), np.array(y_true_f), np.array(y_pred_c), np.array(y_pred_m), np.array(y_pred_f)

In [ ]:
y_true_c, y_true_m, y_true_f, y_pred_c, y_pred_m, y_pred_f = predict_from_pipeline(model, test_dataset_match)

In [ ]:
metrics.lvl_wise_metric([y_true_c, y_true_m, y_true_f],
                        [y_pred_c, y_pred_m, y_pred_f])

In [ ]:
h_measurements,consistency,exact_match = metrics.hmeasurements([y_true_c, y_true_m, y_true_f],
                                                               [y_pred_c, y_pred_m, y_pred_f],
                                                               dataset.get_tree())
print('\nHierarchical Precision =',h_measurements[0],
      '\nHierarchical Recall =', h_measurements[1],
      '\nHierarchical F1-Score =',h_measurements[2],
      '\nConsistency = ', consistency,
      '\nExact Match = ', exact_match,
     )